# Derive insights on Olympics data using Python Pandas 
### <font color='blue'> Expose an integration point using websockets for orchestration with Node-RED.</font>

## 1. Setup
To prepare your environment, you need to install some packages.

### 1.1 Install the necessary packages

You need the latest versions of these packages:<br>
- websocket-client: is a python client for the Websockets.<br>
- python-swiftclient: is a python client for the Swift API.<br><br>

** Install the websocket client: **

In [1]:
!pip install websocket-client

** Install IBM Bluemix Object Storage Client: **

In [2]:
!pip install python-swiftclient

### 1.2 Import packages and libraries

Import the packages and libraries that you'll use:

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import websocket
import thread
import time
import swiftclient
import codecs
from io import StringIO

## 2. Configuration

Add configurable items of the notebook below

### 2.1 Add your service credentials for Object Storage

You must create Object Storage service on Bluemix.
To access data in a file in Object Storage, you need the Object Storage authentication credentials.
Insert the Object Storage authentication credentials as <i><b>credentials_1</b></i> in the following cell after 
removing the current contents in the cell. 

In [4]:
# @hidden_cell
credentials_1 = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_5fb1ce0f_6a6f_431b_8c75_652f4cfdc454',
  'project_id':'63d358127185445ea99ea6f6ba08ef0d',
  'region':'dallas',
  'user_id':'8e32d21adc3d42e3a76dadcc9b63a460',
  'domain_id':'14638fcddb2b424ebfaaa0cc0676944d',
  'domain_name':'1143539',
  'username':'member_20177d52c4d142604aedc2cced3eea1016c751aa',
  'password':"""j0~uJQF^30.ZH(P=""",
  'container':'JourneyTwo',
  'tenantId':'undefined',
  'filename':'olympics.csv'
}


### 2.3 Global Variables

Add global variables.

In [5]:
olympics_data_filename = 'olympics.csv'
dictionary_data_filename = 'dictionary.csv'

# 3. Persistence and Storage

### 3.1 Configure Object Storage Client

In [6]:
auth_url = credentials_1['auth_url']+"/v3"
container = credentials_1["container"]

IBM_Objectstorage_Connection = swiftclient.Connection(
    key=credentials_1['password'], authurl=auth_url, auth_version='3', os_options={
        "project_id": credentials_1['project_id'], "user_id": credentials_1['user_id'], "region_name": credentials_1['region']})

def create_container(container_name):
    """ Create a container on Object Storage.
    """
    x = IBM_Objectstorage_Connection.put_container(container_name)
    return x

def put_object(container_name, fname, contents, content_type):
    """ Write contents to Object Storage.
    """
    x = IBM_Objectstorage_Connection.put_object(
        container_name,
        fname,
        contents,
        content_type)
    return x

def get_object(container_name, fname):
    """ Retrieve contents from Object Storage.
    """
    Object_Store_file_details = IBM_Objectstorage_Connection.get_object(
        container_name, fname)
    return Object_Store_file_details[1]

# 4. Data 

### 4.1 Prepare data
Combine the olympics and dictionary data into a single dataframe:
- Read olympics data from Object Storage.<br>
- Rename columns<br>
- Populate the data in the dictionary to the Olympics data with a merge<br><br>

In [7]:
olympics = pd.read_csv(StringIO(get_object(container, olympics_data_filename).decode('utf-8')))
olympics = olympics.rename(columns = {'NOC':'Code'})
dictionary = pd.read_csv(StringIO(get_object(container, dictionary_data_filename).decode('utf-8')))
olympics = pd.merge(olympics, dictionary, on='Code')
olympics.head()

,City,Edition,Sport,Discipline,Athlete,Code,Gender,Event,Event_gender,Medal,Country,Population,GDP per Capita
0,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold,Hungary,9844686,12363.54346
1,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,1200m freestyle,M,Gold,Hungary,9844686,12363.54346
2,Athens,1896,Athletics,Athletics,"SZOKOLYI, Alajos",HUN,Men,100m,M,Bronze,Hungary,9844686,12363.54346
3,Athens,1896,Athletics,Athletics,"DANI, Nandor",HUN,Men,800m,M,Silver,Hungary,9844686,12363.54346
4,Athens,1896,Athletics,Athletics,"KELLNER, Gyula",HUN,Men,marathon,M,Bronze,Hungary,9844686,12363.54346


# 5. Insights on the data using Python Pandas
- Create re-usable functions

In [8]:
def get_medals_gb_year_country():
    """ Group by edition and country and sum medals count.
    """
    medals_groupedBy_yearCountry = olympics.groupby(['Edition','Code']).apply(lambda country: country['Code'].count())
    return medals_groupedBy_yearCountry

def get_medals_gb_year_country_medal():
    """ Group by edition, country, medal type and sum medals count.
    """
    medals_groupedBy_yearCountryMedal = olympics.groupby(['Edition', 'Code', 'Medal']).apply(lambda country: country['Medal'].count())
    return medals_groupedBy_yearCountryMedal

def get_medals_last_10_years(countrycode):
    """ Get Gold, Silver and Bronze medals for a country for last 10 editions.
    """
    last10pics = olympics['Edition'].unique()
    yrs = pd.Series(last10pics).nlargest(10)
    df = pd.DataFrame([], columns=['Year', 'Gold', 'Silver', 'Bronze'])
    medalsdf = get_medals_gb_year_country_medal()
   
    for yr in yrs:
        medaltally = medalsdf[yr][countrycode]
        gold = 0
        silver = 0
        bronze = 0
        if 'Gold' in medaltally:
            gold = medaltally['Gold']
        if 'Silver' in medaltally:
            silver = medaltally['Silver']
        if 'Bronze' in medaltally:
            bronze =  medaltally['Bronze']
        df1 = pd.DataFrame([[yr,gold, silver, bronze]], columns=['Year', 'Gold', 'Silver', 'Bronze'])
        df = df.append(df1, ignore_index=True) 
    df = df.sort_values(by=['Year'], ascending=True)    
    df = df.reset_index()
    del df['index']
    return df

def get_correlation_medalstally():
    """ Get correlation between the medals tally and population, GDP per capita.
    """
    df = get_medals_gb_year_country()
    values  = get_all_olympic_years().values
    size = values.size
    correlations = []
    for i in range(size):
        year = values[i][0]
        df1 = df[year].to_frame(name="Tally")
        df1 = df1.reset_index()
        df2 = pd.merge(df1,dictionary, on='Code')
        corrpop = df2.corr().values[0][1]
        corrgdp = df2.corr().values[0][2]
        resp = {"Year": year, "Population":corrpop, "GDP":corrgdp}
        correlations.append(resp)
    return correlations  

def get_medals_category(countrycode, year):
    """ Get the medals count in different sports category for a country in an edition.
    """
    df = olympics[olympics['Edition'] ==  year]
    df1 = df[df['Code'] == countrycode]
    df2 = df.groupby(['Sport']).apply(lambda country: country['Medal'].count())
    return df2

def get_medals_category_all(countrycode):  
    """ Get the medals count in different sports category for a country for last ten editions.
    """
    df1 = olympics[olympics['Code'] == countrycode]
    df2 = df1.groupby(['Sport']).apply(lambda country: country['Medal'].count())
    return df2

def get_top_ten_gold_tally(year):
    """ Get the top ten gold medal winning countries in an edition.
    """
    df = olympics[olympics['Edition'] ==  year]
    df1 = df[df['Medal'] == 'Gold']
    df2 = df1.groupby(['Code']).apply(lambda country: country['Medal'].count())
    return df2

def get_top_ten_total_tally(year):
    """ Get the top ten total medal winning countries in an edition.
    """
    df = olympics[olympics['Edition'] ==  year]
    df1 = df.groupby(['Code']).apply(lambda country: country['Medal'].count())
    return df1

def get_year_venue():
    """ Get edition venue matrix.
    """
    df = olympics[['Edition', 'City']]
    df = df.drop_duplicates()
    df = df.reset_index()
    df = df.set_index('Edition')
    del df['index']
    return df.sort_index()

def get_all_olympic_years():
    """ Get list of all olympic editions.
    """
    df = olympics['Edition']
    df = df.drop_duplicates()
    df = df.reset_index()
    del df['index']
    return df.sort_index()

def get_all_countries():
    """ Get list of all countries.
    """
    df = olympics[['Code','Country']]
    df = df.drop_duplicates()
    df = df.reset_index()
    del df['index']
    return df.sort(['Country'],ascending=[True])

def get_country_edition_data(countrycode,edition):
    """ Get data for a country and edition.
    """
    df = olympics[olympics["Code"] == countrycode]
    df1 = df[df["Edition"] == edition]
    return df1

# 6. Expose integration point with a websocket client 

In [12]:
def on_message(ws, message):
    print(message)
    msg = json.loads(message)
    cmd = msg['cmd']
    
    if cmd == 'MBY':
        country = msg['country']
        tally = get_medals_last_10_years(country)    
        tallyarray=[]
        for i, row in tally.iterrows():
            medaltally = {"Year":int(row["Year"]),
                          "Gold":int(row["Gold"]),
                          "Silver":int(row["Silver"]),
                          "Bronze":int(row["Bronze"])}
            tallyarray.append(medaltally)
        wsresponse = {}
        wsresponse["forcmd"] = "MBY" 
        wsresponse["response"] = tallyarray
        ws.send(json.dumps(wsresponse))
    elif cmd == 'MBSC':
        country = msg['country']
        year = 2008
        response = get_medals_category(country, year)
        
        ct = response.count()
        if ct > 5:
            response = response.nlargest(5)    
        
        medals = []
        countries = []
        for i, row in response.iteritems():
            countries.append(i)
            medals.append(row)   
  
        wsresponse = {}
        wsresponse["forcmd"] = "MBSC"
        wsresponse["response"] = { "countries":countries, "medals":medals}         
        ws.send(json.dumps(wsresponse))
    elif cmd == 'MBSA':
        country = msg['country']
        response = get_medals_category_all(country)
        
        ct = response.count()
        if ct > 5:
            response = response.nlargest(5)    
        
        medals = []
        countries = []
        for i, row in response.iteritems():
            countries.append(i)
            medals.append(row)   
  
        wsresponse = {}
        wsresponse["forcmd"] = "MBSA"
        wsresponse["response"] = { "countries":countries, "medals":medals}         
        ws.send(json.dumps(wsresponse))    
    elif cmd == 'T10G':
        edition = msg["edition"]
        response = get_top_ten_gold_tally(edition)
        ct = response.count()
        if ct > 10:
            response = response.nlargest(10)
        medals = []
        for i, row in response.iteritems():
            data = {"country":i,"tally":row}
            medals.append(data)  
        wsresponse = {}
        wsresponse["forcmd"] = "T10G"
        wsresponse["response"] = medals   
        print(wsresponse)
        ws.send(json.dumps(wsresponse))     
    elif cmd == 'T10M':
        year = msg["edition"]
        response = get_top_ten_total_tally(year)
        ct = response.count()
        if ct > 10:
            response = response.nlargest(10)
        medals = []
        for i, row in response.iteritems():
            data = {"country":i,"tally":row}
            medals.append(data)  
        wsresponse = {}
        wsresponse["forcmd"] = "T10M"
        wsresponse["response"] = medals   
        print(wsresponse)
        ws.send(json.dumps(wsresponse)) 
    elif cmd == 'CORR':
        corr = get_correlation_medalstally() 
        wsresponse = {}
        wsresponse["forcmd"] = "CORR"
        wsresponse["response"] = corr
        ws.send(json.dumps(wsresponse)) 
    elif cmd == 'YV':   
        yearvenue = get_year_venue()
        yearvenuearray = []
        for i in range(yearvenue.size):
            value = {"Year":yearvenue.index[i],"Venue":yearvenue.values[i].tolist()[0]}
            yearvenuearray.append(value)
        responsejson = {}
        responsejson["forcmd"]="YV"
        responsejson["response"]=yearvenuearray
        ws.send(json.dumps(responsejson))               
    elif cmd == 'DATA':
        country = msg['country']
        edition = msg['edition']
        olympicsslice = get_country_edition_data(country,edition)
        data = []
        numofcolumns = olympicsslice.columns.size
        cols = []
        values = []
        for column in olympicsslice.columns:
            cols.append(column)
        for value in olympicsslice.values:
            values.append(value.tolist()) 
        data = {"cols":cols,"vals":values}    
        responsejson = {}
        responsejson['forcmd']='DATA'
        responsejson['response']= data
        ws.send(json.dumps(responsejson)) 
    elif cmd == 'EDITIONS':
        years = get_all_olympic_years()
        yearsarray = []
        for i,row in years.iteritems():
            for value in row:
                yearsarray.append(value)
        length = len(yearsarray)
        wsresponse = []
        for i in range(length):
            year = {"text":yearsarray[i],"value":yearsarray[i]}
            wsresponse.append(year)
        responsejson = {}
        responsejson['forcmd']='EDITIONS'
        responsejson['response']= wsresponse 
        ws.send(json.dumps(responsejson)) 
    elif cmd == 'COUNTRIES':
        countries = get_all_countries()
        countriesarray = []
        codearray = []
        for i,row in countries.iteritems():
            if i=='Code':
                for value in row:
                    codearray.append(value)
            elif i=='Country':  
                for value in row:
                    countriesarray.append(value)
        length = len(codearray)
        wsresponse = []
        for i in range(length):
            country = {"text":countriesarray[i],"value":codearray[i]}
            wsresponse.append(country)
        responsejson = {}
        responsejson['forcmd']='COUNTRIES'
        responsejson['response']= wsresponse 
        ws.send(json.dumps(responsejson))  

def on_error(ws, error):
    print(error)

def on_close(ws):
    ws.send("DSX Listen End")

def on_open(ws):
    def run(*args):
        for i in range(10000):
            hbeat = '{"cmd":"Olympics DSX HeartBeat"}'
            ws.send(hbeat)
            time.sleep(100)
            
    thread.start_new_thread(run, ())


def start_websocket_listener():
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp("ws://osstorenodered.mybluemix.net/ws/orchestrate",
                              on_message = on_message,
                              on_error = on_error,
                              on_close = on_close)
    ws.on_open = on_open
    ws.run_forever()

# 7. Start websocket client

In [ ]:
start_websocket_listener()

--- request header ---
GET /ws/orchestrate HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: osstorenodered.mybluemix.net
Origin: http://osstorenodered.mybluemix.net
Sec-WebSocket-Key: AFRGSrbvVFmNkNP0eiwjLA==
Sec-WebSocket-Version: 13


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Connection: Upgrade
Sec-WebSocket-Accept: QOIGB/oj5p/23NVzNyKKBQrAk/Q=
Date: Wed, 09 Aug 2017 04:30:38 GMT
X-Global-Transaction-ID: 1109793655
Upgrade: websocket
-----------------------
send: '\x81\xa0\x80?\x14N\xfb\x1dw#\xe4\x1d.l\xcfSm#\xf0Vw=\xa0{G\x16\xa0wq/\xf2KV+\xe1K63'


{"cmd":"Olympics DSX HeartBeat"}
{"cmd":"CORR"}


send: "\x81\xfe\x08.Z\xe0\xd8\x19!\xc2\xbev(\x83\xb5}x\xda\xf8;\x19\xaf\x8aKx\xcc\xf8;(\x85\xabi5\x8e\xab|x\xda\xf8B!\xc2\x9f]\n\xc2\xe29w\xd0\xf6-o\xd5\xe8*k\xd3\xef(k\xd1\xe8,j\xd6\xef*v\xc0\xfaI5\x90\xadu;\x94\xb1v4\xc2\xe29j\xce\xe9/m\xd2\xeb,j\xd2\xe0)o\xd3\xed)c\xd8\xec5z\xc2\x81|;\x92\xfa#z\xd1\xe0 l\x9d\xf49!\xc2\x9f]\n\xc2\xe29w\xd0\xf6)j\xd0\xef)h\xd5\xe8/h\xd7\xeb*l\xd3\xec.c\xd0\xe05z\xc2\x88v*\x95\xb4x.\x89\xb7wx\xda\xf84j\xce\xe8,n\xd6\xeb!m\xd5\xe8+b\xd0\xe1(m\xd4\xed/v\xc0\xfa@?\x81\xaa;`\xc0\xe9 j\xd0\xa55z\x9b\xfa^\x1e\xb0\xfa#z\xd0\xf6+l\xd0\xe9+c\xd2\xe0 l\xd8\xe8)i\xd5\xe8.v\xc0\xfaI5\x90\xadu;\x94\xb1v4\xc2\xe29j\xce\xe1/l\xd0\xeb!m\xd3\xe1,o\xd4\xed*j\xd5\xef5z\xc2\x81|;\x92\xfa#z\xd1\xe1)n\x9d\xf49!\xc2\x9f]\n\xc2\xe29j\xce\xea+b\xd0\xea+n\xd1\xea,j\xd3\xed!h\xd8\xea5z\xc2\x88v*\x95\xb4x.\x89\xb7wx\xda\xf8)t\xd1\xec,j\xd3\xea!m\xd5\xeb*c\xd4\xe0-j\xd6\xf49x\xb9\xbdx(\xc2\xe29k\xd9\xe8!'\xcc\xf8bx\xa7\x9cIx\xda\xf8)t\xd5\xe9/j\xd6\xed(j\xd7\xec+b\xd1\xeb/i\xd8\xf

{"forcmd": "CORR", "response": [{"GDP": -0.45503137111050673, "Population": 0.16723502805350984, "Year": 1896}, {"GDP": -0.00070250627336347908, "Population": -0.054638750280917456, "Year": 1900}, {"GDP": 0.26012928968003507, "Population": 0.96603873955453057, "Year": 1904}, {"GDP": 0.22802241250358282, "Population": 0.14503287533948406, "Year": 1908}, {"GDP": 0.51606510742813638, "Population": 0.21369353610782776, "Year": 1912}, {"GDP": 0.40248248615620269, "Population": 0.71131969374058557, "Year": 1924}, {"GDP": 0.2820601363517013, "Population": 0.068449396191777853, "Year": 1928}, {"GDP": 0.2391065304845863, "Population": 0.13766293249267933, "Year": 1932}, {"GDP": 0.3207828478028838, "Population": 0.03735565482214076, "Year": 1936}, {"GDP": 0.46318992187419555, "Population": 0.13441389116269073, "Year": 1948}, {"GDP": 0.20558512419096453, "Population": 0.10438078465444142, "Year": 1952}, {"GDP": 0.18610168730691373, "Population": 0.13458884299488039, "Year": 1956}, {"GDP": 0.19584

/usr/local/src/bluemix_jupyter_bundle.v55/notebook/lib/python2.7/site-packages/ipykernel/__main__.py:113: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
send: '\x81\xfe\x12l\xa2\xf9\xdb\x11\xd9\xdb\xbd~\xd0\x9a\xb6u\x80\xc3\xfb3\xe1\xb6\x8e_\xf6\xab\x92T\xf1\xdb\xf71\x80\x8b\xbeb\xd2\x96\xb5b\xc7\xdb\xe11\xf9\x82\xf9e\xc7\x81\xaf3\x98\xd9\xf9P\xc4\x9e\xb3p\xcc\x90\xa8e\xc3\x97\xf9=\x82\xdb\xadp\xce\x8c\xbe3\x98\xd9\xf9P\xe4\xbe\xf9l\x8e\xd9\xa03\xd6\x9c\xa3e\x80\xc3\xfb3\xe3\x95\xbct\xd0\x90\xba3\x8e\xd9\xf9g\xc3\x95\xaet\x80\xc3\xfb3\xe3\xb5\x9c3\xdf\xd5\xfbj\x80\x8d\xbei\xd6\xdb\xe11\x80\xb8\xa9v\xc7\x97\xafx\xcc\x98\xf9=\x82\xdb\xadp\xce\x8c\xbe3\x98\xd9\xf9P\xf0\xbe\xf9l\x8e\xd9\xa03\xd6\x9c\xa3e\x80\xc3\xfb3\xe3\x8b\xb6t\xcc\x90\xba3\x8e\xd9\xf9g\xc3\x95\xaet\x80\xc3\xfb3\xe3\xab\x963\xdf\xd5\xfbj\x80\x8d\xbei\xd6\xdb\xe11\x80\xb8\xaeb\xd6\x8b\xba}\xcb\x98\xf9=\x82\xdb\xadp\xce\x8c\xbe3\x98\xd9\xf9P\xf7\xaa\xf9l\x8e\xd9\xa03\xd6\x9c\xa3e\x80\xc3\xfb3\xe

...

{"cmd":"Olympics DSX HeartBeat"}


send: '\x81\xa0kw\x9a\xf4\x10U\xf9\x99\x0fU\xa0\xd6$\x1b\xe3\x99\x1b\x1e\xf9\x87K3\xc9\xacK?\xff\x95\x19\x03\xd8\x91\n\x03\xb8\x89'


{"cmd":"Olympics DSX HeartBeat"}


send: '\x81\xfe\x01k=Gv\xfeFe\x10\x91O$\x1b\x9a\x1f}V\xdcivF\xb9\x1fkV\xdcO"\x05\x8eR)\x05\x9b\x1f}V\xa5Fe\x15\x91H)\x02\x8cDeL\xde\x1f\x12%\xbf\x1fkV\xdcI&\x1a\x92DeL\xde\x0cuC\x83\x11g\r\xdc^(\x03\x90I5\x0f\xdc\x07gT\xbdu\tT\xd2\x1de\x02\x9fQ+\x0f\xdc\x07gA\xca@kV\x85\x1f$\x19\x8bS3\x04\x87\x1f}V\xdco\x12%\xdc\x11gT\x8a\\+\x1a\x87\x1f}V\xca\x0e:Z\xdeFe\x15\x91H)\x02\x8cDeL\xde\x1f\x003\xac\x1fkV\xdcI&\x1a\x92DeL\xde\tu\x0b\xd2\x1d<T\x9dR2\x18\x8aO>T\xc4\x1de=\xb1oeZ\xde\x1f3\x17\x92Q>T\xc4\x1dsG\x83\x11g\r\xdc^(\x03\x90I5\x0f\xdc\x07gT\xb0x\x03T\xd2\x1de\x02\x9fQ+\x0f\xdc\x07gB\xce@kV\x85\x1f$\x19\x8bS3\x04\x87\x1f}V\xdc|\x12%\xdc\x11gT\x8a\\+\x1a\x87\x1f}V\xcd\x0c:Z\xdeFe\x15\x91H)\x02\x8cDeL\xde\x1f\x004\xac\x1fkV\xdcI&\x1a\x92DeL\xde\x0ev\x0b\xd2\x1d<T\x9dR2\x18\x8aO>T\xc4\x1de0\xac|eZ\xde\x1f3\x17\x92Q>T\xc4\x1duC\x83\x11g\r\xdc^(\x03\x90I5\x0f\xdc\x07gT\xb4m\tT\xd2\x1de\x02\x9fQ+\x0f\xdc\x07gD\xcd@\x1a\x0b'
send: '\x81\xfe\x01o\xb6\xa8 J\xcd\x8aF%\xc4\xcbM.\x94\x92\x00h\xe2\x99\

{"cmd":"Client connected"}
{"cmd":"T10G","edition":2008}
{'forcmd': 'T10G', 'response': [{'country': 'USA', 'tally': 125}, {'country': 'CHN', 'tally': 74}, {'country': 'RUS', 'tally': 43}, {'country': 'GER', 'tally': 42}, {'country': 'KOR', 'tally': 41}, {'country': 'NED', 'tally': 40}, {'country': 'AUS', 'tally': 31}, {'country': 'GBR', 'tally': 31}, {'country': 'FRA', 'tally': 25}, {'country': 'JPN', 'tally': 23}]}
{"cmd":"T10M","edition":2008}
{'forcmd': 'T10M', 'response': [{'country': 'USA', 'tally': 315}, {'country': 'CHN', 'tally': 184}, {'country': 'AUS', 'tally': 149}, {'country': 'RUS', 'tally': 143}, {'country': 'GER', 'tally': 101}, {'country': 'KOR', 'tally': 78}, {'country': 'GBR', 'tally': 77}, {'country': 'FRA', 'tally': 76}, {'country': 'BRA', 'tally': 75}, {'country': 'ESP', 'tally': 71}]}
{"cmd":"MBS","edition":2008,"country":"GER"}
{"cmd":"MBY","country":"USA"}
